# How long is a day?
In this tutorial we will use an astronomical Python library to show how the length of a day varies during one year. Let's start from importing the ephem module and instantiating an observer:

In [1]:
import ephem
observer = ephem.Observer()

To set observer's localization we have to set the position on the Earth's surface:

In [2]:
observer.lat = "51.109454"
observer.lon = "17.032686" # not necessary for this study
observer.elev = 120
print(observer)

<ephem.Observer date='2017/9/27 20:18:22' epoch='2000/1/1 12:00:00' lon='17:01:57.7' lat='51:06:34.0' elevation=120.0m horizon=0:00:00.0 temp=15.0C pressure=1010.0mBar>


We can see that the observer is related with a date when the object was created. We need all days in one year, so a good choice is to use the pandas module and its fixed frequency datetime index for the current year:

In [3]:
import pandas as pd

current_year = observer.date.triple()[0]
all_days = pd.date_range(str(current_year) + "/01/01", str(current_year) + "/12/31")

To calculate the length of a specific day we have to estimate time between the Sun's rising and setting:

In [4]:
sun = ephem.Sun()

for ad in all_days:
    observer.date = ad
    observer.next_setting(sun) - observer.next_rising(sun)